In [ ]:
import numpy as np
import scipy.optimize as optm
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button

import piplite
await piplite.install(["ipywidgets", 'mini-auspex'])
from ipywidgets import FloatSlider, Button
from ipywidgets import interact
from IPython.display import Markdown, display, Image
from smartwedge.smartwedge import Smartwedge

In [2]:
# Calcula as multiplas relfexões
t0 = 0
tf = 120
multiple_reflections = 4


def calc_rapido(wc, c):
    # Especificações do transdutor:
    # Imasonic
    pitch = 0.6e-3  # em mm
    coord = np.zeros((64, 2))
    coord[:, 0] = np.linspace(-63 * pitch / 2, 63 * pitch / 2, 64)
    angles = np.arange(-90, 90).astype(float)

    # Parâmetros da Smartwedge:
    # c = 84.28
    r0 = 70
    espessura = 20e-3 # 20 mm de espessura
    # wc = 6.2
    offset = 2
    Tprime = 19.5

    # Velocidade do som nos materiais em mm/us
    v1 = 6.40
    v2 = 1.483
    v3 = 5.900

    # Criação do objeto smartwedge_old6400,87783467447
    sm = Smartwedge(c, r0, wc, v1, v2, Tprime, offset)

    # focus_angle_vec = list()
    focus_angle_vec = np.zeros_like(angles)

    v1 = v1 * 1e3
    v2 = v2 * 1e3
    v3 = v3 * 1e3

    time_bd = np.zeros_like(angles)
    time_dw = np.zeros_like(angles)
    time_ws = np.zeros_like(angles)

    dists = sm.compute_total_dist(angles)
    focus_angle, _ = sm.compute_entrypoints(angles)

    time_bd = dists[:, 0] / v1
    time_dw = dists[:, 1] / v1
    time_ws = dists[:, 2] / v2

    time_wedge = (time_bd + time_dw) * 2  # tempo total dentro da sm
    time_coluna_agua = ((time_ws * 2) + time_wedge)  # tempo total do transdutor até a sax2.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1))uperficie externa da meia cana

    time_meia_cana = np.zeros_like(time_bd)
    time_meia_cana.fill(espessura/ v3)
    time_meia_cana = time_coluna_agua + (time_meia_cana * 2)  # tempo total do transdutor ate a superficie interna da meia cana

    return time_wedge, time_coluna_agua, time_meia_cana, sm

In [ ]:
c_slider = FloatSlider(
    value=84.28,
    min=60.0,
    max=120.0,
    step=0.01,
    description='C:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)
wc_slider = FloatSlider(
    value=6.20,
    min=2.0,
    max=15.0,
    step=0.01,
    description='wc',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)


@interact()
def c(c=c_slider.value, wc=wc_slider.value):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15,10))
    plt.gca().invert_yaxis()
    ax1.set_ylim(top=t0, bottom=tf)
    ax1.set_xlabel('Ângulo de disparo[°]')
    ax1.set_ylabel('Tempo em $\mu$s')
    fig.subplots_adjust(left=0.25, bottom=0.25)

    time_wedge, time_coluna_agua, time_meia_cana, sm = calc_rapido(wc, c)
    ang = np.arange(-90, 90, 1)

    for reflections in range(1, multiple_reflections + 2, 1):
        ax1.plot(ang, (time_wedge * 1e6) * reflections, 'o', color='blue', lw=2, alpha=0.3)
        ax1.plot(ang, (time_coluna_agua * 1e6) * reflections, 'o', color='magenta', lw=2, alpha=0.3)
        ax1.plot(ang, (time_meia_cana * 1e6) * reflections, 'o', color='green', lw=2, alpha=0.3)

    ax1.set_title(f"Multiplas Reflexões")
   
    ax2.set_title("SmartWedge")

    distancias = sm.compute_total_dist(ang)
    sm_dist = distancias[90][0] * 1e3   # Distância do centro do transdutor até a "lente" em mm
    wc_dist = distancias[90][2] * 1e3   # Distância da "lente" até a superficie externa da tubulação
    ax2.hlines(0, sm.B[0], sm.B[0] - (sm_dist + wc_dist), colors='red',
               label=f'Distância do centro do transdutor até a externa: {sm_dist + wc_dist:.3f}mm')

    ax2.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1))
    sm.draw(plt=ax2, plot_tube=True, lente_color='blue', externa_color='magenta')
    plt.show()    
    
    
    